# カテゴリーデータの処理

現実のデータセットには、往々にしてカテゴリ値の特徴量が含まれている。<br>
1.順序特徴量（シャツのサイズのように、順序が定義されているもの）<br>
2.クラスラベル（正解データが数値ではなく文字列）<br>
2.名義特徴量（色のように、順序が全くないもの<br>
例えば下記のようなデータのときがその例になる。

In [28]:
import pandas as pd
#サンプルデータを生成（Tシャツの色・サイズ・価格・クラスラベル）
df = pd.DataFrame([ \
                   ['green','M',10.1,'class1'], \
                   ['red','L',13.5,'class2'], \
                   ['blue','XL',15.3,'class1']
                  ])
#列名を設定
df.columns = ['color','size','price','classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


## 順序特徴量のマッピング

整数などへの変換マッピングは明示的に定義する必要がある。<br>
例えば下記のようになる<br>

In [29]:
#Tシャツのサイズと整数を対応させるディクショナリを生成
size_mapping = {'XL':3,'L':2,'M':1}
#Tシャツのサイズを整数に変換
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


変換した整数値を元の順序特徴量に戻す場合は、逆のマッピングディレクトリを定義ｓるう。

In [23]:
#整数からTシャツのサイズへ返還するディクショナリ
inv_size_mapping = {v:k for k,v in size_mapping.items()}
df['size'] = df['size'].map(inv_size_mapping)
df
df['size'] = df['size'].map(size_mapping)

## クラスラベルのエンコーディング

推定器のほとんどは、クラスラベルを内部で整数に変換する。<br>
だが、技術的なミスを回避するには、scikit-learning内部でクラスラベルを整数に変換するのではなく、<br>
あらかじめクラスラベルを整数の配列として提供することがよい<br>
方法は次の通り<br>
1.クラスラベルと整数を対応させるディレクトリを定義<br>
2.LabelEncoderを利用する<br>
数値変換したクラスラベルから元のクラスラベルに戻すには、<br>
逆のマッピングディレクトリを定義すればよい

### 対応させるディレクトリを定義する

In [25]:
import numpy as np
#クラスラベルと整数を対応させるディクショナリを生成
class_mapping = {label:idx for idx,label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [30]:
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


In [32]:
inv_class_mapping = {v : k for k,v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


### LabelEncoderを使う

In [35]:
from sklearn.preprocessing import LabelEncoder

#ラベルエンコーダのインスタンスを生成
class_le = LabelEncoder()
#クラスラベルから整数に変換
y = class_le.fit_transform(df['classlabel'].values)
y

array([0, 1, 0], dtype=int64)

In [34]:
#クラスラベルを文字列に戻す
class_le.inverse_transform(y)

array(['class1', 'class2', 'class1'], dtype=object)

## 名義特徴量を変換する

名義特徴量を変換するときには、LabelEncoderは利用できない<br>
LabelEncoderを利用すると、下のようにblue->0,green->1,red->2と順番がついてしまう

In [37]:
#Tシャツの色、サイズ、価格を抽出
X = df[['color','size','price']].values
color_le = LabelEncoder()
X[:,0] = color_le.fit_transform(X[:,0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

その問題を解決するため、one-hotエンコーディングを利用する。<br>
この手法は、名義特徴量の列の一意な値ごとにダミー特徴量を作成するという発想<br>
例えば今回の例では、新たにblue,green,redの3つ特徴量が追加される

In [40]:
pd.get_dummies(df[['price','color','size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


missingValueのカテゴリーデータの処理を持ってくる